In [1]:
import os
import flopy as f
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
ws = "work/ex11"
name = "modflow_project"
exe_name = "exe/mf6.exe"

In [8]:
from shutil import which
print(which(os.path.abspath(exe_name)))

c:\Users\omowumie\OneDrive - University of Delaware - o365\Documents\GitHub\flopy_project\exe\mf6.exe


In [9]:
os.path.abspath(exe_name)

'c:\\Users\\omowumie\\OneDrive - University of Delaware - o365\\Documents\\GitHub\\flopy_project\\exe\\mf6.exe'

In [10]:
os.path.isfile(exe_name)

True

In [11]:
# create simulation
sim = f.mf6.MFSimulation(sim_name=name, sim_ws=ws, exe_name=exe_name)

In [12]:
# Temporal discretization
nper = 1
perlen = 8640000
nstp = 1
tdis = f.mf6.ModflowTdis(sim, time_units="seconds", nper = 1, perioddata=[(0, 8640000, 1)])

In [13]:
ims = f.mf6.ModflowIms(sim)

In [14]:
# create model object
gwf = f.mf6.ModflowGwf(sim, modelname=name)

In [15]:
# Spatial Discretization
nlay, nrow, ncol = 1, 25, 20
delr=delc=50
top=120
botm=75
ibound = np.ones((nlay, nrow, ncol), dtype=np.int64)
ibound[:,:,:7]=0
ibound[:,:8,11]=0

Dis = f.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                            botm=botm,delr=delr, delc=delc, idomain=ibound)

In [16]:
# Node Properties Definition
kh=kva=10e-5
laytyp=[1]

npf = f.mf6.ModflowGwfnpf(gwf,save_specific_discharge=True, icelltype=laytyp, k=kh, k33=kva)

In [17]:
# Defining initial conditions

start=80 * np.ones((nlay,nrow,ncol), dtype=np.float64)
ic = f.mf6.ModflowGwfic(gwf, strt=start)

In [18]:
# Recharge package
rch = f.mf6.ModflowGwfrcha(gwf, recharge=2e-8)

In [19]:
# Defining Constant Head
chd_rec= {0:[[0,1,15,100.0,100.0]]}
chd=f.mf6.ModflowGwfchd(gwf, stress_period_data=chd_rec)


In [20]:
headfile=f"{name}.hds"
head_filerecord=[headfile]
budgetfile=f"{name}.cbb"
budget_filerecord=[budgetfile]
saverecord=[("HEAD","ALL"),("BUDGET","ALL")]
printrecord=[("HEAD","LAST")]

oc=f.mf6.ModflowGwfoc(gwf,saverecord=saverecord,head_filerecord=head_filerecord,budget_filerecord=budget_filerecord,printrecord=printrecord)

In [21]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package ims_-1...
  writing model modflow_project...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package rcha_0...
    writing package chd_0...
INFORMATION: maxbound in ('gwf6', 'chd', 'dimensions') changed to 1 based on size of stress_period_data
    writing package oc...


In [23]:
success, buff = sim.run_simulation()
# assert success, "MODFLOW 6 did not terminate normally"

FloPy is using the following executable to run the model: exe/mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                        VERSION 6.4.1 Release 12/09/2022

   MODFLOW 6 compiled Dec 09 2022 18:18:37 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Government shall b